In [1]:
import os
import pathlib
import re
import copy
import warnings

from sys import getsizeof

import cv2
import xmltodict
import numpy as np
import pandas as pd
import tiffcapture as tc

from tifffile import imsave
from matplotlib import pyplot as pt

%matplotlib inline

In [2]:
POSITION = 40
PROCESS_PI = False
TIME_UNIT_FACTOR = 600
PIXEL_INCH_RATIO = .647
DIAMETER = 15

In [3]:
# path to exit data
exit_dat = '../results/40-47.csv'

# Function Defs

In [7]:
def parse_trees(links_tab, verbose=False):
    """
    Given 'links in tab statistics' from TrackMate.
    Construct data structures representing trees ('tree') and
    their branching ('branch') respectively.
    
    -   'tree' contains mapping of TREE_ID (which is the same as
        the id of root's SPOT_ID) to its associated tree.
        Each tree is in turn a map of TREE_BRANCH_ID
        (which is the branch's first spot's SPOT_ID)
        to its branch which is represented as list of SPOT_ID's.
    -   'branch' encodes how the tree is structured. It contains
        mapping of TREE_ID to the branching configuration.
        The branching configuration is encoded as mapping of
        BRANCH_ID to its two children's BRANCH_ID.
    """
    
    tree = {}
    branch = {}
    for track in links_tab.TRACK_ID.unique():
    
        sub = links_tab[links_tab.TRACK_ID == track]

        this_lines = {}
        this_branches = {}
        stack = []
    
        groot = sub.iloc[0][3]
        print("Parsing tree %s" % groot)
        stack.append(groot)
    
        while len(stack) > 0:
        
            root = stack.pop()
            track = [root]
            if verbose:
                print("Parsing subtree %s" % root)
            nsub = sub[sub.SPOT_SOURCE_ID == root]
        
            while nsub.index.size > 0:
                if nsub.index.size == 1:
                    this = nsub.iloc[0][4]
                    track.append(this)
                    nsub = sub[sub.SPOT_SOURCE_ID == this]
                else:
                    stack.append(nsub.iloc[0][4])
                    stack.append(nsub.iloc[1][4])
                    this_branches[root] = (nsub.iloc[0][4], nsub.iloc[1][4])
                    if verbose:
                        print("breaking")
                    break
                
            this_lines[root] = track
            if verbose:
                print("adding branch %s" % root)
    
        if verbose:
            print("Finishing...")
        tree[groot] = this_lines
        branch[groot] = this_branches
        
    return tree, branch

In [11]:
def extract_values(tree, spots_tab, colnames, verbose=False):
    """
    Extract varoius measurement values from spots statistics and save it
    in format similar to parsed tree. The list of values to be extracted
    from spots statistics is defined in 'colnames' 
    
    The information will be stored in following format:
    map(TYPE:(TREE_ID:TREE_BRANCH:list(VALUES))
    
    """
    
    vals = {}

    for tr in tree:
    
        if verbose:
            print("extracting values for %s" % tr)
    
        for colname in colnames:
        
            val_tree = {}
        
            for br in tree[tr]:
            
                brkeys = tree[tr][br]
                brvals = [sits[sits.ID == x][colname].values[0] for x in brkeys]
            
                val_tree[br] = brvals
            
            if colname not in vals:
                vals[colname] = {}
            
            vals[colname][tr] = val_tree
            
    return vals

# Pipeline

In [9]:
position = 1

if position < 10:
    pos = "0%d" % position
else:
    pos = str(position)

track_path = '../test/eli-new-unsync-bf-%s/out-focus/merged/out/tracked/' % pos

ba = pd.read_csv(os.path.join(track_path, 'Links in tracks statistics.csv'))
sits = pd.read_csv(os.path.join(track_path, 'Spots in tracks statistics.csv'))

In [8]:
tree, branch = parse_trees(ba)

Parsing tree 25654
Parsing tree 15343
Parsing tree 21070
Parsing tree 22388
Parsing tree 24157
Parsing tree 30962
Parsing tree 15647
Parsing tree 17202
Parsing tree 22822
Parsing tree 20022
Parsing tree 33438
Parsing tree 17815
Parsing tree 33351
Parsing tree 35763
Parsing tree 46880
Parsing tree 21869
Parsing tree 22719
Parsing tree 20315
Parsing tree 22881
Parsing tree 17509
Parsing tree 15995
Parsing tree 22376
Parsing tree 37511
Parsing tree 17270
Parsing tree 34874
Parsing tree 31952
Parsing tree 15583
Parsing tree 15554
Parsing tree 18390
Parsing tree 26645
Parsing tree 28745
Parsing tree 32324
Parsing tree 23154
Parsing tree 47942
Parsing tree 18609
Parsing tree 34301
Parsing tree 28854
Parsing tree 21216
Parsing tree 37637
Parsing tree 16804
Parsing tree 35872
Parsing tree 17133
Parsing tree 36774
Parsing tree 17057
Parsing tree 20771
Parsing tree 22393
Parsing tree 28022
Parsing tree 31196
Parsing tree 34989
Parsing tree 17758
Parsing tree 35677
Parsing tree 22507
Parsing tree

In [13]:
svals = extract_values(tree, sits, ['POSITION_T', 'POSITION_X', 'POSITION_Y', 'TOTAL_INTENSITY', 'QUALITY'])